In [3]:
import spacy
import sqlite3
from sqlite3 import Error

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/iramkamdar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2024-03-11 12:57:01.364095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/iramkamdar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
def create_connection(db_file):
    """Create and return a database connection."""
    try:
        conn = sqlite3.connect(db_file)
        print("Connection established.")
        return conn
    except Error as e:
        print(e)
    return None

In [6]:
def initialize_db(conn):
    """Initialize the database with required tables."""
    try:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS Patients (id INTEGER PRIMARY KEY, name TEXT, age INTEGER, sex TEXT,contact_info INTEGER);''')
        c.execute('''CREATE TABLE IF NOT EXISTS Appointments (id INTEGER PRIMARY KEY, patient_id INTEGER, doctor_id INTEGER, date TEXT, time TEXT, status TEXT, FOREIGN KEY(patient_id) REFERENCES Patients(id), FOREIGN KEY(doctor_id) REFERENCES Doctors(id));''')
        c.execute('''CREATE TABLE IF NOT EXISTS Doctors (id INTEGER PRIMARY KEY, name TEXT, specialization TEXT);''')
        c.execute('''CREATE TABLE IF NOT EXISTS Feedback (id INTEGER PRIMARY KEY, appointment_id INTEGER, patient_id INTEGER, comments TEXT, rating INTEGER, FOREIGN KEY(appointment_id) REFERENCES Appointments(id), FOREIGN KEY(patient_id) REFERENCES Patients(id));''')
        print("Database initialized.")
    except Error as e:
        print(e)

In [7]:
def insert_patient(conn, name, age, sex, contact_info):
    """Insert a new patient into the Patients table and return the patient ID."""
    sql = '''INSERT INTO Patients(name, age, sex, contact_info) VALUES(?, ?, ?, ?)'''
    cur = conn.cursor()
    cur.execute(sql, (name, age, sex, contact_info))
    conn.commit()
    return cur.lastrowid

In [8]:
def list_patients(conn):
    """List all patients in the database."""
    sql = 'SELECT id, name, age, sex, contact_info FROM Patients ORDER BY id;'
    try:
        cur = conn.cursor()
        cur.execute(sql)
        print("List of Patients:")
        print(f"{'ID':<5} {'Name':<20} {'Age':<5} {'Sex':<10} {'Contact Info'}")
        print("-" * 60)
        patients = cur.fetchall()
        for patient in patients:
            print(f"{patient[0]:<5} {patient[1]:<20} {patient[2]:<5} {patient[3]:<10} {patient[4]}")
    except Error as e:
        print("An error occurred:", e)


In [9]:
def schedule_appointment(conn, patient_id, doctor_id, date, time):
    """Schedule a new appointment."""
    sql = '''INSERT INTO Appointments(patient_id, doctor_id, date, time, status)
             VALUES(?, ?, ?, ?, 'Scheduled');'''
    try:
        cur = conn.cursor()
        cur.execute(sql, (patient_id, doctor_id, date, time))
        conn.commit()
        print(f"Appointment scheduled successfully for {date} at {time}.")
    except Error as e:
        print("An error occurred while scheduling the appointment:", e)


In [10]:
def process_input(conn, user_input):
    """Process user input, recognizing intents and interacting with the database."""
    doc = nlp(user_input)  # Assuming `nlp` is previously defined and loaded with spaCy
    if "list patients" in user_input.lower():
        list_patients(conn)
    elif "list appointments" in user_input.lower():
        list_scheduled_appointments(conn)
    elif "schedule appointment" in user_input.lower():
        print("Let's schedule your appointment.")
        patient_name = input("Enter your name: ")
        age = input("Enter your age: ")
        sex = input("Enter your sex (M/F/Other): ")
        contact_info = input("Enter your contact info: ")
        
        # Insert the patient and get their unique ID
        patient_id = insert_patient(conn, patient_name, age, sex, contact_info)
        
        # Proceed with the scheduling process
        process_input_for_scheduling(conn, patient_id)
    
    elif "add doctor" in user_input.lower():
        print("Let's add a new doctor.")
        doctor_name = input("Enter the doctor's name: ")
        specialization = input("Enter the doctor's specialization: ")
        
        # Insert the doctor into the database
        insert_doctor(conn, doctor_name, specialization)

    elif "feedback" in user_input.lower():
        print("Recognized intent to provide feedback.")
        # Code to implement feedback submission goes here
    
    else:
        print("Sorry, I didn't understand that.")



In [11]:
def process_input_for_scheduling(conn, patient_id):
    """Overall process for scheduling an appointment."""
    doctors = list_doctors(conn)
    if not doctors:
        print("Sorry, no doctors are available at the moment.")
        return
    
    doctor_id = select_doctor(doctors)
    date = input("Enter the date for the appointment (YYYY-MM-DD): ")
    
    if not check_availability_and_schedule(conn, doctor_id, date, patient_id):
        # Handle the case where no slots are available as needed
        pass


In [12]:
def list_scheduled_appointments(conn):
    """List all scheduled appointments."""
    sql = '''SELECT Appointments.id, Patients.name, Doctors.name, Appointments.date, Appointments.time, Appointments.status
             FROM Appointments
             JOIN Patients ON Appointments.patient_id = Patients.id
             JOIN Doctors ON Appointments.doctor_id = Doctors.id
             WHERE Appointments.status = 'Scheduled'
             ORDER BY Appointments.date, Appointments.time;'''
    try:
        cur = conn.cursor()
        cur.execute(sql)
        appointments = cur.fetchall()

        print("\nScheduled Appointments:")
        for appt in appointments:
            print(f"Appointment ID: {appt[0]}, Patient: {appt[1]}, Doctor: {appt[2]}, Date: {appt[3]}, Time: {appt[4]}, Status: {appt[5]}")
    except Error as e:
        print("An error occurred:", e)
        

In [13]:
def insert_doctor(conn, name, specialization):
    """Insert a new doctor into the Doctors table."""
    sql = '''INSERT INTO Doctors(name, specialization) VALUES(?, ?);'''
    cur = conn.cursor()
    cur.execute(sql, (name, specialization))
    conn.commit()
    print(f"Doctor {name} added successfully.")


In [14]:
def list_doctors(conn):
    """List all available doctors."""
    sql = 'SELECT id, name, specialization FROM Doctors ORDER BY id;'
    try:
        cur = conn.cursor()
        cur.execute(sql)
        doctors = cur.fetchall()
        print("List of Doctors:")
        for doctor in doctors:
            print(f"ID: {doctor[0]}, Name: {doctor[1]}, Specialization: {doctor[2]}")
        return doctors
    except Error as e:
        print("An error occurred:", e)
        return []

In [15]:
def select_doctor(doctors):
    """Let the patient select a doctor by ID."""
    doctor_ids = [str(doctor[0]) for doctor in doctors]
    selected_id = input("Enter the ID of the doctor you want to book an appointment with: ")
    while selected_id not in doctor_ids:
        print("Invalid ID. Please select a valid doctor ID.")
        selected_id = input("Enter the ID of the doctor you want to book an appointment with: ")
    return selected_id


In [16]:
def check_availability_and_schedule(conn, doctor_id, date, patient_id):
    """Check availability for the selected doctor on the chosen date and schedule if possible."""
    # Check existing appointments for the doctor on the chosen date
    sql = 'SELECT time FROM Appointments WHERE doctor_id = ? AND date = ? ORDER BY time;'
    cur = conn.cursor()
    cur.execute(sql, (doctor_id, date))
    appointments = cur.fetchall()
    
    if len(appointments) >= 8:  # Assuming a doctor can have 8 appointments a day
        print("No available slots for this day. Please book for another day.")
        return False
    
    # List available slots (assuming appointments are 1 hour each, for simplicity)
    all_slots = ["09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00"]
    booked_slots = [appt[0] for appt in appointments]
    available_slots = [slot for slot in all_slots if slot not in booked_slots]
    
    print("Available slots:")
    for slot in available_slots:
        print(slot)
    
    selected_time = input("Select a time from the available slots: ")
    while selected_time not in available_slots:
        print("Invalid selection. Please select a valid time slot.")
        selected_time = input("Select a time from the available slots: ")
    
    # Schedule the appointment
    # Example of a correct call
    schedule_appointment(conn, patient_id, doctor_id, date, selected_time)

    return True


In [17]:
def show_available_slots(conn, date):
    """Show available slots for all doctors on a given date."""
    doctors = list_doctors(conn)
    available_slots_by_doctor = {}
    for doctor in doctors:
        doctor_id = doctor[0]
        print(f"\nChecking available slots for {doctor[1]} (ID: {doctor_id})...")
        slots = check_availability(conn, doctor_id, date)
        if slots:
            print("Available slots:", ", ".join(slots))
            available_slots_by_doctor[doctor_id] = slots
        else:
            print("No available slots for this doctor.")
    return available_slots_by_doctor


In [18]:
def complete_booking(conn, patient_id, available_slots_by_doctor):
    """Let the patient choose a doctor and an available slot for booking."""
    doctor_id = input("\nEnter the ID of the doctor you want to book with: ")
    if doctor_id not in available_slots_by_doctor:
        print("Invalid doctor ID.")
        return
    
    selected_slot = input("Enter the slot you want to book (HH:MM): ")
    if selected_slot not in available_slots_by_doctor[doctor_id]:
        print("Invalid or unavailable slot.")
        return
    
    # Proceed with booking the appointment
    date = input("For which date do you want to book the appointment? (YYYY-MM-DD): ")  # Assume the date is asked again or carried over
    if schedule_appointment(conn, patient_id, doctor_id, date, selected_slot):
        print("Appointment booked successfully.")
    else:
        print("Failed to book the appointment.")


In [19]:

def main():
    db_file = "hospital_management_system.db"
    conn = create_connection(db_file)
    if conn:
        initialize_db(conn)
        while True:
            user_input = input("How can I assist you today? (type 'exit' to quit): ")
            if user_input.lower() == 'exit':
                break
            process_input(conn, user_input)
        conn.close()
    else:
        print("Failed to create database connection.")

if __name__ == "__main__":
    main()


Connection established.
Database initialized.
Let's schedule your appointment.
List of Doctors:
ID: 1, Name: Dr.Gulam Ali Kamdar, Specialization: Opthalmologist
ID: 2, Name: Dr. Rajesvi, Specialization: Gastroenterologist 
ID: 3, Name: Dr.Gufrab, Specialization: Pulmonologist
ID: 4, Name: Dr.Caroline, Specialization: Cardiologist
Available slots:
09:00
10:00
11:00
12:00
13:00
14:00
15:00
16:00
Appointment scheduled successfully for 2024-04-23 at 10:00.


In [20]:
db_file = "hospital_management_system.db"
conn = create_connection(db_file)
if conn:
     initialize_db(conn)
else:
    print("Failed to create database connection.")

Connection established.
Database initialized.


In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output


In [24]:
 # Widgets for adding a doctor
doctor_name_input = widgets.Text(
    value='',
    placeholder='Enter doctor\'s name',
    description='Name:',
    disabled=False
)

doctor_specialization_input = widgets.Text(
    value='',
    placeholder='Enter specialization',
    description='Specialization:',
    disabled=False
)

add_doctor_button = widgets.Button(description="Add Doctor")

def on_add_doctor_button_clicked(b):
    # Assuming `insert_doctor` function is correctly defined and `conn` is your database connection
    insert_doctor(conn, doctor_name_input.value, doctor_specialization_input.value)
    with output:
        clear_output()
        print(f"Doctor {doctor_name_input.value} added successfully.")

add_doctor_button.on_click(on_add_doctor_button_clicked)

output = widgets.Output()

display(doctor_name_input, doctor_specialization_input, add_doctor_button, output)


Text(value='', description='Name:', placeholder="Enter doctor's name")

Text(value='', description='Specialization:', placeholder='Enter specialization')

Button(description='Add Doctor', style=ButtonStyle())

Output()

In [22]:
def chat_interface():
    chat_output = widgets.Output()

    def process_query(b):
        with chat_output:
            clear_output()
            user_query = text_input.value.lower()
            if "list doctors" in user_query:
                # Assume list_doctors function is defined
                list_doctors(conn)  # Make sure conn is your active SQLite connection
            elif "schedule appointment" in user_query:
                display(appointment_interface())
            # Add more conditions for different functionalities
            else:
                print("How can I help you?")

    text_input = widgets.Text(description="You: ", placeholder="Type here...")
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(process_query)

    display(text_input, submit_button, chat_output)


In [39]:
def appointment_interface(conn):
    # Widgets for patient details
    patient_name_input = widgets.Text(
        description='Patient Name:',
        placeholder='Enter your name'
    )
    
    patient_age_input = widgets.IntText(
        description='Age:',
        placeholder='Enter your age',
        min=0  # Ensure age cannot be negative
    )
    
    patient_sex_dropdown = widgets.Dropdown(
        options=[('Select your sex', ''), ('Male', 'M'), ('Female', 'F'), ('Other', 'O')],
        description='Sex:',
    )
    
    # Widget for selecting a doctor
    doctor_dropdown = widgets.Dropdown(
        options=[("Select a doctor", 0)] + get_doctors(conn),
        description="Doctor:"
    )
    
    # Widget for choosing a date
    date_picker = widgets.DatePicker(
        description='Pick a Date',
        disabled=False
    )
    
    # Widget for selecting a time, initially empty
    time_dropdown = widgets.Dropdown(
        options=[],
        description='Time:',
    )
    
    # Update available time slots when doctor and date are selected
    def update_available_times(*args):
        # Placeholder function for fetching available times based on doctor and date
        # For demonstration, let's assume all slots are available
        time_dropdown.options = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00']
        
    doctor_dropdown.observe(update_available_times, 'value')
    date_picker.observe(update_available_times, 'value')
    
    # Output widget to display messages
    output = widgets.Output()
    
    # Button to submit the appointment request
    submit_button = widgets.Button(description="Schedule")
    
    def on_submit(b):
        with output:
            clear_output()
            if doctor_dropdown.value == 0:
                print("Please select a doctor.")
                return
            # Additional validation for patient details can be added here
            print(f"Appointment scheduled for {patient_name_input.value} ({patient_sex_dropdown.value}, {patient_age_input.value} years old) with Doctor ID {doctor_dropdown.value} on {date_picker.value} at {time_dropdown.value}.")
            # Insert logic for scheduling the appointment in the database
    
    submit_button.on_click(on_submit)
    
    display(patient_name_input, patient_age_input, patient_sex_dropdown, doctor_dropdown, date_picker, time_dropdown, submit_button, output)

# Assuming get_doctors is defined to fetch doctors from your database
    
conn = sqlite3.connect('hospital_management_system.db')

# Call the interface function with the database connection
appointment_interface(conn)


List of Doctors:
ID: 1, Name: Dr.Gulam Ali Kamdar, Specialization: Opthalmologist
ID: 2, Name: Dr. Rajesvi, Specialization: Gastroenterologist 
ID: 3, Name: Dr.Gufrab, Specialization: Pulmonologist
ID: 4, Name: Dr.Caroline, Specialization: Cardiologist


Text(value='', description='Patient Name:', placeholder='Enter your name')

IntText(value=0, description='Age:')

Dropdown(description='Sex:', options=(('Select your sex', ''), ('Male', 'M'), ('Female', 'F'), ('Other', 'O'))…

Dropdown(description='Doctor:', options=(('Select a doctor', 0), (1, 'Dr.Gulam Ali Kamdar', 'Opthalmologist'),…

DatePicker(value=None, description='Pick a Date', step=1)

Dropdown(description='Time:', options=(), value=None)

Button(description='Schedule', style=ButtonStyle())

Output()